# organize csv files

In [ ]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv("./raw_data.csv")
df

In [17]:
for i in range(len(df)):
    if df["case_id"].iloc[i][0] == "B":
        _, code, year = df["case_id"].iloc[i].split("-")
        assert int(year) < 21 and int(year) > 10
        df["case_id"].iloc[i] = year + "-" + code
    else:
        code, year = df["case_id"].iloc[i].split("-")
        if len(year) > 2:
            code, year = year, code
        assert int(year) < 21 and int(year) > 11
        df["case_id"].iloc[i] = year + "-" + code
print(len(df[df["case_id"].duplicated(False)]))
np.unique([i.split("-")[0] for i in df["case_id"].values])

0


/tmp/ipykernel_5849/2022630649.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["case_id"].iloc[i] = year + "-" + code
/tmp/ipykernel_5849/2022630649.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["case_id"].iloc[i] = year + "-" + code


array(['11', '12', '13', '14', '15', '16', '17', '18', '20'], dtype='<U2')

## other subtypes

In [18]:
df[df["subtype"] != "0"]

,subtype,case_id,case_id.1,tx_ends,birth,tanı_tarihi,exitus,exorfu_date,remission,recurrence,recorfu_date
120,endometrioid/teratom,17-6709,B-6709-17,NaN,01.10.1980,17.04.2015,0,11.12.2019,1.0,0.0,11.12.2019
121,cc,17-6551,B-6551-17,18.08.2017,25.08.1968,12.04.2017,0,25.03.2021,1.0,0.0,25.03.2021
122,endometriyozis zemininde seromüsinöz,17-4924,B-4924-17,NaN,26.10.1982,17.03.2017,1,24.02.2019,0.0,0.0,16.11.2017
123,"Seromüsinöz, mülleriant tip mikst epitelyal ka...",17-903,B-903-17,29.05.2017,17.10.1988,13.01.2017,0,27.03.2023,1.0,0.0,01.06.2020
130,cc,18-3341,B-3341-18,11.07.2018,17.01.1977,06.02.2018,1,21.02.2019,0.0,0.0,NaN
131,endometrioid,18-2798,B-2798-18,22.06.2018,27.12.1978,07.02.2018,0,24.10.2018,NaN,0.0,24.10.2018
133,mikst,18-4798,B-4798-18,25.07.2018,28.08.1953,08.03.2018,1,21.03.2022,1.0,1.0,16.01.2020
136,mikst,18-5964,B-5964-18,07.08.2018,11.07.1967,22.03.2018,1,27.09.2018,1.0,1.0,31.08.2018
138,mikst,16-6396,B-6396-16,NaN,06.03.1957,07.04.2016,1,02.11.2018,NaN,0.0,03.09.2018
142,cc,18-6137,B-6137-18,03.08.2018,15.01.1962,29.03.2018,1,27.12.2019,1.0,1.0,12.02.2019


In [19]:
other_st = df[df["subtype"] != "0"]
other_st_ids = other_st["case_id"].values
df = df.drop(other_st.index)
df.shape, len(other_st_ids)

((137, 11), 11)

In [20]:
ffpe_dir = "/media/nfs/SURV/BasOVER/Slides/"
# frozen_dir = "/media/nfs/SURV/BasOVER/frozen/"
other_sites_dir = "/media/nfs/SURV/BasOVER/excluded/other_sites/"

ffpe_list = [i for i in os.listdir(ffpe_dir) if "ndpi" in i if "F" not in i]
ffpe_cases = np.unique([i.rsplit("-", 1)[0] for i in ffpe_list])
print("FFPE case/slide:", len(ffpe_cases), len(ffpe_list))
frozen_list = [i for i in os.listdir(ffpe_dir) if "ndpi" in i if "F" in i]
frozen_cases = np.unique([i.rsplit("-", 1)[0] for i in frozen_list])
print("Frozen case/slide:", len(frozen_cases), len(frozen_list))
total_cases = np.unique(list(frozen_cases)+list(ffpe_cases))
total_list = np.unique(list(frozen_list)+list(ffpe_list))
print("Total case/slide:", len(total_cases), len(total_list))

# bütün görüntülerin follow-up verisi var
assert len([i for i in ffpe_cases if i not in df["case_id"].values]) == 0 and len([i for i in frozen_cases if i not in df["case_id"].values]) == 0
both_ffpe_frozen_cases = [i for i in frozen_cases if i in ffpe_cases]
ffpe_only = [i for i in ffpe_list if i.rsplit("-", 1)[0] in both_ffpe_frozen_cases]
frozen_only = [i for i in frozen_list if i.rsplit("-", 1)[0] in both_ffpe_frozen_cases]
print("Both FFPE and Frozen olan case / only ffpe and only frozen slides: ", len(both_ffpe_frozen_cases), len(ffpe_only), len(frozen_only))

other_site_list = [i for i in os.listdir(other_sites_dir) if "ndpi" in i]
other_site_cases = np.unique([i.rsplit("-", 1)[0] for i in other_site_list])
print("Other sites total: ", len(other_site_cases), len(other_site_list))
removed_other_site_cases = [i for i in other_site_cases if i not in total_cases]
print("Sadece non-ovarian dokusu olanlar:", len(removed_other_site_cases))
[i for i in other_site_list if i.rsplit("-", 1)[0] in removed_other_site_cases]

FFPE case/slide: 122 828
Frozen case/slide: 13 101
Total case/slide: 134 929
Both FFPE and Frozen olan case / only ffpe and only frozen slides:  1 5 1
Other sites total:  6 38
Sadece non-ovarian dokusu olanlar: 3


['14-4493-LA1.ndpi',
 '14-4493-LA5.ndpi',
 '14-2087-RA4.ndpi',
 '14-2087-RA8.ndpi',
 '14-16314-RA3.ndpi',
 '14-2087-LA5.ndpi',
 '14-16314-RA11.ndpi',
 '14-16314-RA7.ndpi',
 '14-2087-LA1.ndpi',
 '14-4493-RA5.ndpi',
 '14-16314-RA2.ndpi',
 '14-2087-LA4.ndpi',
 '14-16314-RA10.ndpi',
 '14-4493-RA1.ndpi',
 '14-16314-RA6.ndpi',
 '14-2087-LA8.ndpi',
 '14-4493-LA4.ndpi',
 '14-2087-RA5.ndpi',
 '14-2087-LA6.ndpi',
 '14-16314-RA4.ndpi',
 '14-2087-RA3.ndpi',
 '14-4493-LA2.ndpi',
 '14-16314-RA8.ndpi',
 '14-2087-RA7.ndpi',
 '14-4493-LA3.ndpi',
 '14-16314-RA9.ndpi',
 '14-2087-RA6.ndpi',
 '14-16314-RA1.ndpi',
 '14-2087-LA7.ndpi',
 '14-4493-RA6.ndpi',
 '14-16314-RA5.ndpi',
 '14-2087-LA3.ndpi',
 '14-4493-RA2.ndpi']

In [21]:
df = df[~df["case_id"].isin(removed_other_site_cases)].reset_index(drop=True)
df.shape

(134, 11)

In [22]:
date_cols = ["birth", "tanı_tarihi", "exorfu_date", "recorfu_date", "tx_ends"]
print("Columns: ", date_cols)
print("Max months: ", [(df[col].str.split(".").str[1]).dropna().astype(int).max() for col in date_cols])
print("Max years: ", [(df[col].str.split(".").str[2]).dropna().astype(int).max() for col in date_cols])
print("Min years: ", [(df[col].str.split(".").str[2]).dropna().astype(int).min() for col in date_cols])


Columns:  ['birth', 'tanı_tarihi', 'exorfu_date', 'recorfu_date', 'tx_ends']
Max months:  [12, 12, 12, 12, 12]
Max years:  [1981, 2018, 2023, 2023, 2021]
Min years:  [1933, 2011, 2011, 2011, 2011]


In [23]:
print("Tanı tarihi:", pd.to_datetime(df["tanı_tarihi"], format="%d.%m.%Y").min(), pd.to_datetime(df["tanı_tarihi"], format="%d.%m.%Y").max())
print("Doğum tarihi:", pd.to_datetime(df["birth"], format="%d.%m.%Y").min(), pd.to_datetime(df["birth"], format="%d.%m.%Y").max())
print("Son tedavi tarihi:", pd.to_datetime(df["tx_ends"], format="%d.%m.%Y").min(), pd.to_datetime(df["tx_ends"], format="%d.%m.%Y").max())

Tanı tarihi: 2011-05-19 00:00:00 2018-04-09 00:00:00
Doğum tarihi: 1933-03-12 00:00:00 1981-09-25 00:00:00
Son tedavi tarihi: 2011-07-11 00:00:00 2021-07-26 00:00:00


In [24]:
df["age"] = (pd.to_datetime(df["tanı_tarihi"], format="%d.%m.%Y") - pd.to_datetime(df["birth"], format="%d.%m.%Y")).dt.days//365
df["age"].min(), df["age"].max()

(34, 84)

In [25]:
df["os_date"] = np.zeros(len(df))
for i in range(len(df)):
    if int(df["exitus"].iloc[i]) == 1:
        df["os_date"].iloc[i] = df["exorfu_date"].iloc[i]
    else:
        df["os_date"].iloc[i] = "02.08.2023"
df[df["os_date"] == 0]

/tmp/ipykernel_5849/432501363.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["os_date"].iloc[i] = "02.08.2023"


,subtype,case_id,case_id.1,tx_ends,birth,tanı_tarihi,exitus,exorfu_date,remission,recurrence,recorfu_date,age,os_date


In [26]:
pd.to_datetime(df["os_date"], format="%d.%m.%Y").min(), pd.to_datetime(df["os_date"], format="%d.%m.%Y").max()

(Timestamp('2012-11-19 00:00:00'), Timestamp('2023-08-02 00:00:00'))

In [27]:
df["os_months"] = (pd.to_datetime(df["os_date"], format="%d.%m.%Y") - pd.to_datetime(df["tanı_tarihi"], format="%d.%m.%Y")).dt.days
df["os_months"] = (df["os_months"]/30).round(2)
df["os_months"].min(), df["os_months"].max()

(1.67, 148.2)

In [29]:
df["dfs_months"] = (pd.to_datetime(df["recorfu_date"], format="%d.%m.%Y") - pd.to_datetime(df["tx_ends"], format="%d.%m.%Y")).dt.days
df["dfs_months"] = (df["dfs_months"]/30).round(2)
df["dfs_months"].min(), df["dfs_months"].max()

(0.0, 144.47)

In [30]:
df["dfs_months"][df["dfs_months"] == 0] = pd.NA
df["dfs_months"].min(), df["dfs_months"].max()

/tmp/ipykernel_5849/2103648142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["dfs_months"][df["dfs_months"] == 0] = pd.NA


(0.4, 144.47)

In [31]:
df

,subtype,case_id,case_id.1,tx_ends,birth,tanı_tarihi,exitus,exorfu_date,remission,recurrence,recorfu_date,age,os_date,os_months,dfs_months
0,0,11-4785,B-4785-11,07.09.2011,01.09.1974,31.05.2011,0,11.05.2023,1.0,0.0,11.05.2023,36,02.08.2023,148.20,142.13
1,0,11-7072,B-7072-11,11.07.2011,01.12.1942,28.06.2011,0,23.05.2023,1.0,0.0,23.05.2023,68,02.08.2023,147.27,144.47
2,0,11-13471,B-13471-11,NaN,10.03.1948,29.11.2011,0,29.11.2011,NaN,NaN,29.11.2011,63,02.08.2023,142.13,NaN
3,0,11-9522,B-9522-11,28.12.2011,10.01.1942,22.08.2011,1,29.10.2016,1.0,1.0,18.10.2012,69,29.10.2016,63.17,9.83
4,0,11-7007,B-7007-11,02.11.2011,17.05.1962,24.06.2011,1,18.07.2018,1.0,1.0,24.11.2011,49,18.07.2018,86.03,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0,18-6609,B-6609-18,27.08.2018,05.04.1960,06.04.2018,1,06.09.2021,1.0,1.0,07.02.2020,58,06.09.2021,41.63,17.63
130,0,18-6152,B-6152-18,26.12.2019,28.11.1962,01.12.2017,1,15.05.2020,1.0,1.0,05.05.2020,55,15.05.2020,29.87,4.37
131,0,18-7055,B-7055-18,NaN,03.04.1966,09.04.2018,1,09.06.2018,0.0,0.0,NaN,52,09.06.2018,2.03,NaN
132,0,18-7241,B-7241-18,27.08.2018,24.03.1972,07.04.2018,0,02.08.2023,1.0,1.0,18.03.2020,46,02.08.2023,64.77,18.97


In [32]:
df.isna().sum()

subtype          0
case_id          0
case_id.1        0
tx_ends         47
birth            0
tanı_tarihi      0
exitus           0
exorfu_date      0
remission       10
recurrence      15
recorfu_date    35
age              0
os_date          0
os_months        0
dfs_months      58
dtype: int64

In [33]:
df.to_csv("./processed_data.csv", index=False)

In [34]:
df["exitus"].value_counts(), df["exitus"].value_counts() / len(df)

(1    87
 0    47
 Name: exitus, dtype: int64,
 1    0.649254
 0    0.350746
 Name: exitus, dtype: float64)

In [36]:
df["os_months"].describe()

count    134.000000
mean      62.658060
std       36.143492
min        1.670000
25%       32.970000
50%       57.500000
75%       93.152500
max      148.200000
Name: os_months, dtype: float64

In [37]:
df["age"].describe() 

count    134.000000
mean      57.723881
std       11.630522
min       34.000000
25%       49.250000
50%       57.000000
75%       68.000000
max       84.000000
Name: age, dtype: float64

In [38]:
dfs_df = df[~df["dfs_months"].isna()]
dfs_df.isna().sum()

subtype         0
case_id         0
case_id.1       0
tx_ends         0
birth           0
tanı_tarihi     0
exitus          0
exorfu_date     0
remission       0
recurrence      0
recorfu_date    0
age             0
os_date         0
os_months       0
dfs_months      0
dtype: int64

In [44]:
dfs_df["recurrence"].value_counts(), dfs_df["recurrence"].value_counts()/len(dfs_df)

(1.0    62
 0.0    14
 Name: recurrence, dtype: int64,
 1.0    0.815789
 0.0    0.184211
 Name: recurrence, dtype: float64)

In [45]:
dfs_df["dfs_months"].describe() 

count     76.000000
mean      25.640263
std       29.213954
min        0.400000
25%        7.125000
50%       17.250000
75%       35.410000
max      144.470000
Name: dfs_months, dtype: float64

# save dataset_csv

In [46]:
import os
import numpy as np
import pandas as pd
df = pd.read_csv("./processed_data.csv")
slide_dir = "/media/nfs/SURV/BasOVER/Slides/"
slide_df = pd.DataFrame({
    'slide_id': [i.rsplit(".", 1)[0] for i in os.listdir(slide_dir)],
    "case_id": [i.rsplit("-", 1)[0] for i in os.listdir(slide_dir)]
})
df = pd.merge(df, slide_df, on="case_id")
print(df.shape, len(df["case_id"].unique()))

os_df = df[["case_id", "slide_id", "age", "exitus", "os_months"]]
dfs_df = df[["case_id", "slide_id", "age", "recurrence", "dfs_months"]]
df

(929, 16) 134


,subtype,case_id,case_id.1,tx_ends,birth,tanı_tarihi,exitus,exorfu_date,remission,recurrence,recorfu_date,age,os_date,os_months,dfs_months,slide_id
0,0,11-4785,B-4785-11,07.09.2011,01.09.1974,31.05.2011,0,11.05.2023,1.0,0.0,11.05.2023,36,02.08.2023,148.20,142.13,11-4785-R02
1,0,11-7072,B-7072-11,11.07.2011,01.12.1942,28.06.2011,0,23.05.2023,1.0,0.0,23.05.2023,68,02.08.2023,147.27,144.47,11-7072-R01
2,0,11-7072,B-7072-11,11.07.2011,01.12.1942,28.06.2011,0,23.05.2023,1.0,0.0,23.05.2023,68,02.08.2023,147.27,144.47,11-7072-R02
3,0,11-7072,B-7072-11,11.07.2011,01.12.1942,28.06.2011,0,23.05.2023,1.0,0.0,23.05.2023,68,02.08.2023,147.27,144.47,11-7072-R03
4,0,11-13471,B-13471-11,NaN,10.03.1948,29.11.2011,0,29.11.2011,NaN,NaN,29.11.2011,63,02.08.2023,142.13,NaN,11-13471-L03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,0,18-6616,B-6616-18,15.08.2018,04.02.1950,05.04.2018,0,06.05.2023,1.0,1.0,26.01.2021,68,02.08.2023,64.83,29.83,18-6616-LO2
925,0,18-6616,B-6616-18,15.08.2018,04.02.1950,05.04.2018,0,06.05.2023,1.0,1.0,26.01.2021,68,02.08.2023,64.83,29.83,18-6616-LO1
926,0,18-6616,B-6616-18,15.08.2018,04.02.1950,05.04.2018,0,06.05.2023,1.0,1.0,26.01.2021,68,02.08.2023,64.83,29.83,18-6616-RO4
927,0,18-6616,B-6616-18,15.08.2018,04.02.1950,05.04.2018,0,06.05.2023,1.0,1.0,26.01.2021,68,02.08.2023,64.83,29.83,18-6616-RO5


In [47]:
dfs_df = dfs_df[~dfs_df["dfs_months"].isna()]
dfs_df.shape, dfs_df.isna().sum()

((513, 5),
 case_id       0
 slide_id      0
 age           0
 recurrence    0
 dfs_months    0
 dtype: int64)

In [48]:
os_df.shape, os_df.isna().sum()

((929, 5),
 case_id      0
 slide_id     0
 age          0
 exitus       0
 os_months    0
 dtype: int64)

In [49]:
os_df["group"] = np.zeros(len(os_df))
dfs_df["group"] = np.zeros(len(dfs_df))

/tmp/ipykernel_5849/2135862653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  os_df["group"] = np.zeros(len(os_df))


In [50]:
os_df.rename(columns={
    'os_months':'survival_months',
    "exitus": "event"
}).to_csv("../../datasets_csv/baskent.csv", index=False)

dfs_df.rename(columns={
    'dfs_months':'survival_months',
    "recurrence": "event"
}).to_csv("../../datasets_csv/baskent_dfs.csv", index=False)

In [51]:
os_df["censorship"] = 1 - os_df["exitus"]
dfs_df["censorship"] = 1 - dfs_df["recurrence"]
os_df["censorship"].value_counts(), dfs_df["censorship"].value_counts()

/tmp/ipykernel_5849/4030651114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  os_df["censorship"] = 1 - os_df["exitus"]


(0    545
 1    384
 Name: censorship, dtype: int64,
 0.0    411
 1.0    102
 Name: censorship, dtype: int64)

In [52]:
os_df.drop(columns=["group", 'exitus']).rename(columns={
    'os_months':'survival_months'
}).to_csv("../../../PORPOISE-master/datasets_csv/baskent.csv", index=False)

dfs_df.drop(columns=["group", 'recurrence']).rename(columns={
    'dfs_months':'survival_months'
}).to_csv("../../../PORPOISE-master/datasets_csv/baskent_dfs.csv", index=False)

In [9]:
import pandas as pd
os_df = pd.read_csv("../../datasets_csv/baskent_dfs.csv")
os_df["survival_months"].describe()

count    641.000000
mean      31.567566
std       25.976836
min        0.470000
25%       13.700000
50%       25.530000
75%       40.130000
max      145.430000
Name: survival_months, dtype: float64

In [10]:
os_df.drop_duplicates("case_id")["survival_months"].describe()

count     98.000000
mean      30.373163
std       27.719446
min        0.470000
25%       12.417500
50%       22.500000
75%       34.860000
max      145.430000
Name: survival_months, dtype: float64